In [3]:
import scanpy as sc
# sc.__version__
import sys
# print(sys.executable)

import scanpy as sc
ad.settings.allow_write_nullable_strings = True

import os
os.makedirs("results/figures", exist_ok=True)

/Users/medha/miniconda3/envs/scrna/bin/python


In [16]:
data_dir = "data/processed"
adata = sc.read_10x_mtx(data_dir, var_names="gene_symbols", cache=False)  # :contentReference[oaicite:9]{index=9}
adata.var_names_make_unique()

# QC
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

# Filtering (typical starting thresholds; adjust)
adata = adata[adata.obs.n_genes_by_counts > 200, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]
sc.pp.filter_genes(adata, min_cells=3)

# Normalize + HVGs + PCA
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)

# Neighbors + clustering + UMAP
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)
sc.tl.leiden(adata, resolution=0.6)
sc.tl.umap(adata)

# Markers
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")

# Save
adata.write("data/processed/pbmc3k_scanpy.h5ad")

/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/scanpy/preprocessing/_simple.py:293: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number
/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/scanpy/preprocessing/_scale.py:309: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/medha/miniconda3/envs/scrna/lib/python3.11/functools.py:909: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)
/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/scanpy/tools/_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(


In [17]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
sc.pl.violin(
    adata,
    keys=["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
    show=False,
    save="_qc_violin.png"
)

In [21]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)
sc.tl.leiden(adata, resolution=0.6)
sc.tl.umap(adata)

sc.pl.umap(
    adata,
    color=["leiden"],
    show=False,
    save="_umap_leiden.png"
)


sc.pl.umap(adata, color=["pct_counts_mt", "total_counts"], show=False, save="_umap_qc.png")

[<Axes: title={'center': 'pct_counts_mt'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <Axes: title={'center': 'total_counts'}, xlabel='UMAP1', ylabel='UMAP2'>]

In [22]:
sc.tl.rank_genes_groups(adata, groupby="leiden", method="wilcoxon")

sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden",
    n_genes=5,          # top 5 per cluster
    show=False,
    save="_dotplot_top5_markers.png"
)


/Users/medha/miniconda3/envs/scrna/lib/python3.11/site-packages/scanpy/tools/_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(


{'mainplot_ax': <Axes: >,
 'group_extra_ax': <Axes: >,
 'gene_group_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

In [25]:
#### dotplot of cannonical PBMC markers

pbmc_markers = {
    "NK cells": ["NKG7", "GNLY"],
    "B cells": [ "CD79A"],
    "Monocytes": ["LYZ", "S100A8", "S100A9"],
    "DC": ["FCER1A", "CST3"],
}

sc.pl.dotplot(
    adata,
    var_names=pbmc_markers,
    groupby="leiden",
    show=False,
    save="_dotplot_canonical_markers.png"
)

{'mainplot_ax': <Axes: >,
 'gene_group_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

In [24]:
print(adata.var_names[:5])
print(adata.var.columns)

Index(['TNFRSF18', 'CPSF3L', 'ATAD3C', 'RER1', 'TNFRSF25'], dtype='str')
Index(['gene_ids', 'mt', 'n_cells_by_counts', 'mean_counts',
       'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts',
       'log1p_total_counts', 'n_cells', 'highly_variable', 'means',
       'dispersions', 'dispersions_norm', 'mean', 'std'],
      dtype='str')
